# Load and Preprocess the JSON Data


In [5]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Load the JSON file
with open("knowledgebase.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Initialize SBERT model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Prepare data for embeddings
texts = []
metadata = []

for chapter, topics in data.items():
    for topic_title, topic_content in topics.items():
        # Append main topic
        texts.append(f"{topic_title}: {topic_content}")
        metadata.append({"title": topic_title, "chapter": chapter})

# Convert texts to embeddings
embeddings = model.encode(texts, convert_to_numpy=True)

# Verify the number of embeddings
print(f"Number of embeddings: {embeddings.shape[0]}")


Number of embeddings: 112


# Store Embeddings in FAISS

In [6]:
# Define the FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance (Euclidean)
index.add(embeddings)  # Add embeddings to index

# Save FAISS index and metadata for retrieval
faiss.write_index(index, "textbook_faiss.index")

# Save metadata
with open("metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4)


# Testing Retrieval

In [9]:
def search(query, top_k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, top_k)

    results = []
    for i in range(len(indices[0])):
        idx = indices[0][i]
        results.append({
            "title": metadata[idx]["title"],  # Topic title
            "chapter": metadata[idx]["chapter"],  # Chapter name
            "score": distances[0][i]
        })

    return results

# Example query
query = "Right-Hand Thumb Rule"
results = search(query)

for res in results:
    print(res)


{'title': '13.2.2 Right-Hand Thumb Rule', 'chapter': '13 CHAPTER', 'score': 0.9642046}
{'title': 'name', 'chapter': '7 CHAPTER', 'score': 1.54195}
{'title': '13.4 ELECTRIC MOTOR', 'chapter': '13 CHAPTER', 'score': 1.5689719}


In [17]:
import json

# Load full knowledge base (JSON textbook)
with open("knowledgebase.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def get_explanation(query, top_k=1):
    results = search(query, top_k)
    
    if not results:
        return "No relevant information found."
    
    best_match = results[0]  # Get the top-ranked result
    best_title = best_match["title"]
    best_chapter = best_match["chapter"]

    # Search JSON structure for the matching content
    if best_chapter in data:
        for topic_title, topic_content in data[best_chapter].items():
            if topic_title == best_title:
                return topic_content  # Return the matching topic's content

    return "No relevant information found."

# Test the function
query = "Right-Hand Thumb Rule"
explanation = get_explanation(query)
print("Explanation:", explanation)


Explanation:  A convenient way of finding the direction of magnetic field associated with a current-carrying conductor is given in Fig. 13.7. Variable resistance (a) (b) Figure 13.6 (a) A pattern of concentric circles indicating the field lines of a magnetic field around a straight conducting wire. The arrows in the circles show the direction of the field lines. (b) A close up of the pattern obtained. Magnetic Effects of Electric Current 121 ©KTBS Not to be republished Imagine that you are holding a current-carrying straight conductor in your right hand such that the thumb points towards the direction of current. Then your fingers will wrap around the conductor in the direction of the field lines of the magnetic field, as shown in Fig. 13.7. This is known as the right-hand thumb rule*. Figure 13.7 Right-hand thumb rule 1. Example 13.1 A current through a horizontal power line flows in east to west direction. What is the direction of magnetic field at a point directly below it and at a 

In [ ]:
            "13.2.2 Right-Hand Thumb Rule":" A convenient way of finding the direction of magnetic field associated with a current-carrying conductor is given in Fig. 13.7. Variable resistance (a) (b) Figure 13.6 (a) A pattern of concentric circles indicating the field lines of a magnetic field around a straight conducting wire. The arrows in the circles show the direction of the field lines. (b) A close up of the pattern obtained. Magnetic Effects of Electric Current 121 ©KTBS Not to be republished Imagine that you are holding a current-carrying straight conductor in your right hand such that the thumb points towards the direction of current. Then your fingers will wrap around the conductor in the direction of the field lines of the magnetic field, as shown in Fig. 13.7. This is known as the right-hand thumb rule*. Figure 13.7 Right-hand thumb rule 1. Example 13.1 A current through a horizontal power line flows in east to west direction. What is the direction of magnetic field at a point directly below it and at a point directly above it? Solution Solution Solution The current is in the east-west direction. Applying the right-hand thumb rule, we get that the magnetic field (at any point below or above the wire) turns clockwise in a plane perpendicular to the wire, when viewed from the east end, and anti-clockwise, when viewed from the west end",
